In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST
from torchvision.transforms import transforms
from tqdm import tqdm
import warnings
warnings.filterwarnings(action="ignore")

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
])

In [3]:
# defining our training and test sets
train_dataset = FashionMNIST(root=".", train=True, download=True, transform=transform)
test_dataset = FashionMNIST(root=".", train=False, download=True, transform=transform)
# creating our dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [4]:
classes = ["T-shirt/Top", "Trouser", "Pullover", "Dress", "Coat",  "Sandal",  "Shirt", "Sneaker", "Bag", "Ankle Boot"]

$$\textsf{Convolution:} \left\lfloor \frac{D_{i}-D_{f}+2p}{s} \right\rfloor + 1 \quad \quad \quad \textsf{Pooling:} \left\lfloor \frac{D_{i}}{D_{f}} \right\rfloor$$

$$D_{i} = \textsf{Dimension of input}; \quad D_{f} = \textsf{Dimension of filter (kernel)}; \quad p = \textsf{Padding}; \quad s = \textsf{Stride}$$

In [5]:
# https://datascience.stackexchange.com/a/64281
# https://stats.stackexchange.com/a/292064
# https://arxiv.org/pdf/1603.07285v1.pdf

In [6]:
# Image properties:
# (Width, Height, Channel) = (28, 28, 1)
# Channel: (1 for grayscale) or (3 for RGB)

In [7]:
device = torch.device('mps')

In [8]:
class Dimension():
    def __init__(self, size, RGB=False):
        self.__size:int = size
        self.__in_channels:int = 3 if RGB else 1
        self.printDimension('Inital tensor size')

    def printDimension(self, text="Tensor size:"):
        print(f"{text}: {self.__size}x{self.__size}x{self.__in_channels}")
   
    def Convolution(self, out_channels:int, kernel_size:int, stride=1, padding=0):
        self.__in_channels = out_channels
        self.__size = ((self.__size - kernel_size + 2*padding) // stride) + 1
        self.printDimension('After convolution')
        return self
   
    def Pooling(self, kernel_size:int):
        self.__size = self.__size // kernel_size
        self.printDimension('After pooling')
        return self


In [9]:
image_size = 28 # 28x28
Prepocessing = Dimension(size=image_size, RGB=False)
Prepocessing.Convolution(32, kernel_size=5).Pooling(2).Convolution(64, kernel_size=5).Pooling(2)

Inital tensor size: 28x28x1
After convolution: 24x24x32
After pooling: 12x12x32
After convolution: 8x8x64
After pooling: 4x4x64


In [10]:
class ConvNet(nn.Module):
    def __init__(self):
        # call the parent constructor
        super(ConvNet, self).__init__()
                
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5)
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5)
        self.pool2 = nn.MaxPool2d(2)

        # Fully connected layers
        self.flatten = nn.Flatten() # input tensor have to be flattened 
        self.fc1 = nn.Linear(in_features=4*4*64, out_features=256)
        self.drop1 = nn.Dropout(p=0.3)
        self.fc2 = nn.Linear(in_features=256, out_features=64)
        self.drop2 = nn.Dropout(p=0.3)
        self.fc3 = nn.Linear(in_features=64, out_features=10)
      
    def forward(self, x):
        # Pass through sequence-1
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        # Pass through sequence-2
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        # Pass through classifier
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.drop1(x)
        x = F.relu(self.fc2(x))
        x = self.drop2(x)
        x = F.relu(self.fc3(x))
        x = F.log_softmax(x, dim=1)
        return x

In [11]:
model = ConvNet()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [12]:
num_epochs = 10

for epoch in range(num_epochs):
    # Training the model ===============
    model.train()
    train_loss = 0.0

    for inputs, targets in tqdm(train_loader):
        inputs = inputs.to(device)
        targets = targets.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Compute average training loss
    train_loss /= len(train_loader)

    # Testing the model ===============
    model.eval()
    test_loss = 0.0
    test_correct = 0

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()

    # Compute average test loss
    test_loss /= len(test_loader)

    # Compute test accuracy
    test_accuracy = 100.0 - test_loss

    # Print training and testing statistics
    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Loss: {train_loss:.4f}, "
          f"Test Accuracy: {test_accuracy:.2f}%")

100%|██████████| 938/938 [00:11<00:00, 80.09it/s]


Epoch [1/10], Train Loss: 0.6798, Test Accuracy: 99.61%


100%|██████████| 938/938 [00:11<00:00, 83.49it/s]


Epoch [2/10], Train Loss: 0.3694, Test Accuracy: 99.68%


100%|██████████| 938/938 [00:11<00:00, 83.00it/s]


...

Epoch [8/10], Train Loss: 0.2064, Test Accuracy: 99.73%


100%|██████████| 938/938 [00:10<00:00, 85.83it/s]


Epoch [9/10], Train Loss: 0.1942, Test Accuracy: 99.74%


100%|██████████| 938/938 [00:11<00:00, 82.50it/s]


Epoch [10/10], Train Loss: 0.1804, Test Accuracy: 99.73%
